# Alpaca-py crypto trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with crypto trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for crypto as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

api_key = "PKU2S8Q3PPWPVRP6FNCL"
secret_key = "5HqA8cPDarpVtSMBrLrcd5s2sUSbEo5iVFgbYVgP"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
# install alpaca-py
! python3 -m pip install alpaca-py

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [3]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.data.live.crypto import *
from alpaca.data.historical.crypto import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError

In [ ]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

'0.21.2'

In [4]:
# check version of alpaca-py
alpaca.__version__

'0.8.2'

# Trading Client

In [5]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [ ]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

In [16]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [17]:
# get list of crypto pairs
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  asset_class=AssetClass.CRYPTO,
  status=AssetStatus.ACTIVE
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': False,
     'id': UUID('bce5b2e7-0678-4b81-9607-4a997471fc92'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 35248.501938667,
     'min_trade_increment': 1e-09,
     'name': 'Shib USD Tether',
     'price_increment': 1e-08,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'SHIB/USDT',
     'tradable': True},
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('da533b7e-5b35-45ef-8ba8-4dd5d44978b8'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 45871.559633027,
     'min_trade_increment': 1e-09,
     'name': 'Shiba Inu / USD Coin',
   

### Orders

In [18]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder


# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [ ]:
# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = symbol,
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

In [ ]:
# simple, market order, notional

req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

In [37]:
# trailing Stop Loss
from alpaca.trading.requests import TrailingStopOrderRequest



req = TrailingStopOrderRequest(
    symbol = symbol,
    notional = 10,
    side = OrderSide.BUY,
    time_in_force = TimeInForce.DAY,
    trail_price = 6.15)

res = trade_client.submit_order(req)
res


APIError: {"message": "forbidden."}


In [22]:
# simple, limit order
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 60000,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '97334c66-5c42-4618-93a3-e9aaed1279d4',
    'created_at': datetime.datetime(2024, 6, 13, 4, 48, 39, 654783, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('3648c904-3901-4a5d-b10e-860dac02697f'),
    'legs': None,
    'limit_price': '60000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 6, 13, 4, 48, 39, 654783, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',

In [23]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 0.01,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 55000,
                    stop_price = 60000
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '7c68d4ff-6b33-4e31-bf98-a8fcdb9c39d5',
    'created_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('7f5e4eaf-9d9d-4ab7-8154-a12c3413ba3f'),
    'legs': None,
    'limit_price': '55000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '60000',
    'submitted_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',
    

In [24]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '7c68d4ff-6b33-4e31-bf98-a8fcdb9c39d5',
     'created_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('7f5e4eaf-9d9d-4ab7-8154-a12c3413ba3f'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
   

In [25]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.CLOSED,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '7c68d4ff-6b33-4e31-bf98-a8fcdb9c39d5',
     'created_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('7f5e4eaf-9d9d-4ab7-8154-a12c3413ba3f'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': datetime.datetime(2024, 6, 13, 4, 48, 46, 60575, tzinfo=TzInfo(UTC)),
   

In [ ]:
# cancel all open orders
trade_client.cancel_orders()

### Positions

In [6]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

APIError: {"message": "forbidden."}


In [ ]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [31]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

keyboard interrupt, bye


# Market Data (Historical)

In [32]:
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()

In [33]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_bars(req).df

open        high         low  \
symbol  timestamp                                                     
BTC/USD 2024-06-08 05:00:00+00:00  69278.70  69321.1725  69197.5175   
        2024-06-08 06:00:00+00:00  69294.95  69399.2400  69238.4000   

                                       close    volume  trade_count  \
symbol  timestamp                                                     
BTC/USD 2024-06-08 05:00:00+00:00  69288.090  0.000706          3.0   
        2024-06-08 06:00:00+00:00  69313.235  0.000056          4.0   

                                           vwap  
symbol  timestamp                                
BTC/USD 2024-06-08 05:00:00+00:00  69298.881613  
        2024-06-08 06:00:00+00:00  69300.108549

In [34]:
# get historical trades by symbol
req = CryptoTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
crypto_historical_data_client.get_crypto_trades(req).df

price      size  \
symbol  timestamp                                              
BTC/USD 2024-06-08 05:34:49.206164+00:00  69307.99  0.000283   
        2024-06-08 05:47:56.991273+00:00  69296.61  0.000283   

                                                           id  
symbol  timestamp                                              
BTC/USD 2024-06-08 05:34:49.206164+00:00  3896482511384563408  
        2024-06-08 05:47:56.991273+00:00  7096718593807172272

In [35]:
# get latest quotes by symbol
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res

{'BTC/USD': {   'ask_exchange': None,
     'ask_price': 67490.2,
     'ask_size': 0.2802,
     'bid_exchange': None,
     'bid_price': 67402.77,
     'bid_size': 0.278652,
     'conditions': None,
     'symbol': 'BTC/USD',
     'tape': None,
     'timestamp': datetime.datetime(2024, 6, 13, 4, 47, 37, 85373, tzinfo=TzInfo(UTC))}}